In [666]:
import pandas as pd 
df_train = pd.read_csv(r'../train.tsv',sep='\t')
df_test = pd.read_csv(r'../test.tsv',sep='\t')

-----------------------------------------------------------
-----------------------------------------------------------
I aim to test my work using only 100,000 records to minimize modeling time until I achieve a promising model. Therefore, this section of the code is temporary.

In [667]:
import numpy as np

np.random.seed(42)

df = df_train.sample(n=100000)

df2 = df_test.sample(n=100000)


df.reset_index(drop=True, inplace=True)

df2.reset_index(drop=True, inplace=True)

In [668]:
# df.shape
# df.info()
df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,777341,F/ship 4 Totoro Washi + 1 pen,1,Handmade/Paper Goods/Stationery,NaN,12.0,1,This listing is for all 4 Totoro washi tape fo...
1,1463629,UCLA Men's Bundle + Shorts,1,Women/Other/Other,Adidas,76.0,1,7 items. 1: XL. 2: 2XL. 3:2XL. 4: XL. 5: 2XL. ...
2,350669,Listing for lol,1,Beauty/Makeup/Lips,NaN,12.0,1,- sunglasses and necklace :)
3,310222,25 pcs kawaii sticker flakes,1,Kids/Toys/Arts & Crafts,NaN,3.0,1,I ordered a bunch of stickers so you will reci...
4,759257,Chanel Mini Lipgloss Set,2,Beauty/Makeup/Lips,Chanel,30.0,1,Brand new never used authentic Mini Lipgloss g...


-----------------------------------------------------------
-----------------------------------------------------------
these functions in preprocessing_pipeline for : <br>
1- drop item description <br>
2- train id columns<br>
3- brand_pipeline ---> replace nulls with other<br>
note :We should not replace NaN values with the modes of brands because null brands should be taken into consideration. This is important as some people may not know the brand of the item.<br>
4- filter_nonzero_price

In [669]:
# Drop the 'item_description' column from the DataFrame
def drop_item_description(df):
    if 'item_description' in df.columns:
        df = df.drop('item_description', axis=1)
    return df

In [670]:
# Drop the 'train_id' column from the DataFrame
def drop_train_id(df):
    if 'train_id' in df.columns:
        df = df.drop('train_id', axis=1)
    elif 'test_id' in df.columns :
        df = df.drop('test_id', axis=1)
    return df

In [671]:
def brand_pipeline(df) : 
    df["brand_name"].fillna("other", inplace=True)
    return df

In [672]:
def filter_nonzero_price(df):
    if 'train_id' in df.columns:
        df = df[df['price'] != 0]
    return df 


In [673]:
def preprocessing_pipeline(df):
    df = filter_nonzero_price(df)
    df = drop_train_id(df)
    df = drop_item_description(df)
    df = brand_pipeline(df)
    return df

In [674]:
df = preprocessing_pipeline(df)
df2 = preprocessing_pipeline(df2)

final version of data before encoding

In [675]:
df.head(10)

,name,item_condition_id,category_name,brand_name,price,shipping
0,F/ship 4 Totoro Washi + 1 pen,1,Handmade/Paper Goods/Stationery,other,12.0,1
1,UCLA Men's Bundle + Shorts,1,Women/Other/Other,Adidas,76.0,1
2,Listing for lol,1,Beauty/Makeup/Lips,other,12.0,1
3,25 pcs kawaii sticker flakes,1,Kids/Toys/Arts & Crafts,other,3.0,1
4,Chanel Mini Lipgloss Set,2,Beauty/Makeup/Lips,Chanel,30.0,1
5,Maroon Foamposites,3,Men/Shoes/Fashion Sneakers,Nike,225.0,1
6,INC studdedHeart Black Blouse Dolman,2,Women/Tops & Blouses/Blouse,INC International Concepts,16.0,1
7,Leggo silicone molds,3,Home/Kitchen & Dining/Bakeware,other,12.0,0
8,Supreme Uzi Chain,1,Handmade/Accessories/Men,other,15.0,1
9,Women Gold Palm Pendant Necklace FC,1,Vintage & Collectibles/Jewelry/Necklace,other,17.0,1


In [676]:
df2.head(10)

,name,item_condition_id,category_name,brand_name,shipping
0,LuLaRoe TC Owls,1,Women/Jeans/Leggings,LuLaRoe,1
1,Htc desire 626,2,Electronics/Cell Phones & Accessories/Cell Pho...,HTC,0
2,Vs Pink zip up jacket with hood,3,Women/Sweaters/Full Zip,PINK,1
3,Women's Adidas Cloudfoam Shoes,2,Women/Shoes/Athletic,Adidas,0
4,Vs pink pants,2,Women/Athletic Apparel/Tracksuits & Sweats,PINK,0
5,"New Case MacBook Air 13""",1,"Electronics/Cell Phones & Accessories/Cases, C...",other,1
6,Hunter Boots,3,Kids/Other/Other,other,0
7,Miss me shirts sz 30.,3,"Women/Jeans/Capri, Cropped",Miss Me,1
8,Tupperware Chef Series Fondue Set,3,Home/Kitchen & Dining/Kitchen Utensils & Gadgets,Tupperware,0
9,•FREE SHIP• PINK (L) Bling Varsity Tee,1,Women/Tops & Blouses/T-Shirts,PINK,1


these functions in feature_engineering_and_encoding_pipeline is for : <br>
1- sub_categories ---> to add 'category1', 'category2', 'category3' instead of category_name<br>
2- add_name_score ---> it generates a score out of 10 for each name

In [677]:
def sub_categories(df) : 
    df[['category1', 'category2', 'category3']] = df['category_name'].str.split('/', expand=True, n=2)
    df = df.drop('category_name', axis=1)
    return df

In [678]:
from textblob import TextBlob
import nltk
nltk.download('punkt')

# Download the 'punkt' resource
nltk.download('punkt')

def calculate_name_score(name, weights):
    name = name.lower()
    
    # Calculate the ease of pronunciation (using TextBlob's sentiment analysis)
    pronunciation_score = TextBlob(name).sentiment.polarity

    def calculate_beauty_and_word_count_score(text):
        # Create a TextBlob object from the text
        blob = TextBlob(text)
    
        # Get the sentiment polarity score (range from -1 to 1, where >0 indicates positive sentiment)
        sentiment_score = blob.sentiment.polarity

        target_word_count = 3  # Adjust this to your preference
        word_count = len(blob.words)
        word_count_difference = abs(target_word_count - word_count)
        word_count_score = 1 / (1 + word_count_difference)
    
        # Calculate beauty and cleverness scores
        beauty_score = sentiment_score
        word_count_score = word_count_score
    
        return beauty_score, word_count_score


    # Calculate beauty and cleverness scores
    beauty_score, num_words_score = calculate_beauty_and_word_count_score(name)
    
    # Calculate the total score
    total_score = (
        weights['num_words'] * num_words_score +
        weights['pronunciation'] * pronunciation_score +
        weights['beauty'] * beauty_score
    )
    
    return total_score

def add_name_score(df) : 
    weights = {
        'num_words': 3.0,
        'pronunciation': 4.0,
        'beauty': 3.0,
    }
    df['name_score'] = df['name'].apply(lambda x: calculate_name_score(x, weights))
    return df 


[nltk_data] Downloading package punkt to C:\Users\MSI GE76
[nltk_data]     Raider\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\MSI GE76
[nltk_data]     Raider\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [679]:
def add_brand_score_for_train_data(df):
    global brand_scores_dict
    # Calculate the average price for each brand
    brand_avg_prices = df.groupby('brand_name')['price'].mean()
    
    # Normalize the average prices to a scale of 0 to 10
    min_avg_price = brand_avg_prices.min()
    max_avg_price = brand_avg_prices.max()
    brand_avg_prices_normalized = 10 * (brand_avg_prices - min_avg_price) / (max_avg_price - min_avg_price)
    
    # Create a dictionary mapping brands to their normalized scores
    brand_scores_dict = brand_avg_prices_normalized.to_dict()
    
    # Add a new 'brand_score' column to the DataFrame based on the brand's score
    df['brand_score'] = df['brand_name'].map(brand_scores_dict)
    
    return df

for test dataset thesr is no price, so we can't calculate the score of barnd, because of that we reuse the values calculated with train dataset<br>
if the brand is not exsist in train data we find the score of it in the same way which used for name with small changes

In [680]:
def add_brand_score_for_test_data(df, brand_scores_dict):
    brand_scores = []
    for brand in df['brand_name']:
        if brand in brand_scores_dict:
            brand_scores.append(brand_scores_dict[brand])
        else:
            weights = {
                'num_words': 3.0,
                'pronunciation': 4.0,
                'beauty': 3.0,
            }
            brand_score = calculate_brand_score(brand, weights)
            brand_scores.append(brand_score)    
    df['brand_score'] = brand_scores
    return df

# from here it is a helper method only
from textblob import TextBlob
import nltk
nltk.download('punkt')

# Download the 'punkt' resource
nltk.download('punkt')

def calculate_brand_score(name, weights):
    name = name.lower()
    
    # Calculate the ease of pronunciation (using TextBlob's sentiment analysis)
    pronunciation_score = TextBlob(name).sentiment.polarity

    def calculate_beauty_and_word_count_score_for_brand(text):
        # Create a TextBlob object from the text
        blob = TextBlob(text)
    
        # Get the sentiment polarity score (range from -1 to 1, where >0 indicates positive sentiment)
        sentiment_score = blob.sentiment.polarity

        target_word_count = 1  # Adjust this to your preference
        word_count = len(blob.words)
        word_count_difference = abs(target_word_count - word_count)
        word_count_score = 1 / (1 + word_count_difference)
    
        # Calculate beauty and cleverness scores
        beauty_score = sentiment_score
        word_count_score = word_count_score
    
        return beauty_score, word_count_score


    # Calculate beauty and cleverness scores
    beauty_score, num_words_score = calculate_beauty_and_word_count_score_for_brand(name)
    
    # Calculate the total score
    total_score = (
        weights['num_words'] * num_words_score +
        weights['pronunciation'] * pronunciation_score +
        weights['beauty'] * beauty_score
    )
    
    return total_score


[nltk_data] Downloading package punkt to C:\Users\MSI GE76
[nltk_data]     Raider\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\MSI GE76
[nltk_data]     Raider\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [681]:
def replace_item_condition_id_with_avg_price_for_train_dataset(df):
    global avg_prices_dict  # Use the global variable
    
    # Calculate the average price for each 'item_condition_id'
    avg_prices = df.groupby('item_condition_id')['price'].mean()

    # Normalize the average prices to a scale of 10
    min_avg_price = avg_prices.min()
    max_avg_price = avg_prices.max()
    avg_prices_normalized = 10 * (avg_prices - min_avg_price) / (max_avg_price - min_avg_price)

    # Create a dictionary to map 'item_condition_id' to scaled average prices
    avg_prices_dict = avg_prices_normalized.to_dict()
    print(avg_prices_dict)
    
    # Replace 'item_condition_id' values with their scaled average prices
    df['item_condition_id_score'] = df['item_condition_id'].map(avg_prices_dict)

    return df

# Print the global avg_prices_dict
print(avg_prices_dict)


{1: 2.4499138988034623, 2: 3.4144498971521524, 3: 2.7745456348049453, 4: 0.0, 5: 10.0}


In [682]:
def replace_item_condition_id_with_avg_price_for_test_dataset(df, avg_prices_dict):
    # Replace 'item_condition_id' values with their scaled average prices using the provided dictionary
    df['item_condition_id_score'] = df['item_condition_id'].map(avg_prices_dict)

    return df

In [683]:
def category_encoder_for_train_data_set(df) : 
    global category1_sum_prices
    global category2_sum_prices
    global category3_sum_prices
    
    # Create dictionaries to store the sums of prices for each unique value
    category1_avg_prices = df.groupby('category1')['price'].mean()
    category2_avg_prices = df.groupby('category2')['price'].mean()
    category3_avg_prices = df.groupby('category3')['price'].mean()

    # Normalize the average prices to a scale of 10
    min_avg_price_cat1 = category1_avg_prices.min()
    max_avg_price_cat1 = category1_avg_prices.max()
    avg_prices_normalized_cat1 = 10 * (category1_avg_prices - min_avg_price_cat1) / (max_avg_price_cat1 - min_avg_price_cat1)

    min_avg_price_cat2 = category2_avg_prices.min()
    max_avg_price_cat2 = category2_avg_prices.max()
    avg_prices_normalized_cat2 = 10 * (category2_avg_prices - min_avg_price_cat2) / (max_avg_price_cat2 - min_avg_price_cat2)

    min_avg_price_cat3 = category3_avg_prices.min()
    max_avg_price_cat3 = category3_avg_prices.max()
    avg_prices_normalized_cat3 = 10 * (category3_avg_prices - min_avg_price_cat3) / (max_avg_price_cat3 - min_avg_price_cat3)

    category1_sum_prices = avg_prices_normalized_cat1.to_dict()
    category2_sum_prices = avg_prices_normalized_cat2.to_dict()
    category3_sum_prices = avg_prices_normalized_cat3.to_dict()

    # Replace the category values with scaled numeric values
    df['category1_score'] = df['category1'].map(category1_sum_prices)
    df['category2_score'] = df['category2'].map(category2_sum_prices)
    df['category3_score'] = df['category3'].map(category3_sum_prices)
    return df


print(category1_sum_prices)
print(category2_sum_prices)
print(category3_sum_prices)


{'Beauty': 0.6164387063588429, 'Electronics': 10.0, 'Handmade': 0.0, 'Home': 3.3834527412676354, 'Kids': 1.2077619645956925, 'Men': 8.875585732823748, 'Other': 1.9355308626882544, 'Sports & Outdoors': 3.533299237918996, 'Vintage & Collectibles': 5.9579697963843, 'Women': 5.910398924687974}
{'Accessories': 1.7094435810161832, 'Antique': 2.190694615501379, 'Apparel': 1.2358823049979775, 'Art': 1.1571772356607568, 'Artwork': 0.9245689737627332, 'Athletic Apparel': 2.7496972455111277, 'Automotive': 2.212260115099207, 'Bags and Purses': 7.762453520729115, 'Bath': 1.3982558264234146, 'Bath & Body': 1.5876029695849183, 'Bathing & Skin Care': 1.1283423453930665, 'Bedding': 3.2562449801578235, 'Blazers & Sport Coats': 4.779383061869688, 'Book': 1.2357067964141375, 'Books': 1.2677897655400938, 'Books and Zines': 0.0, 'Boys (4+)': 2.635341677138568, 'Boys 0-24 Mos': 1.4113932552458741, 'Boys 2T-5T': 1.6218059429246474, 'Cameras & Photography': 7.558674815376329, 'Candles': 2.3384623303977796, 'Ca

In [684]:
def category_encoder_for_test_data_set(df):
    # Replace the category values with their corresponding values in the dictionary
    df['category1_score'] = df['category1'].map(category1_sum_prices).fillna(category1_sum_prices['Other'])
    df['category2_score'] = df['category2'].map(category2_sum_prices).fillna(category2_sum_prices['Other'])
    df['category3_score'] = df['category3'].map(category3_sum_prices).fillna(category3_sum_prices['Other'])
    
    return df

In [685]:
def feature_engineering_and_encoding_pipeline(df) : 
    df = sub_categories(df)
    df = add_name_score(df)
    df = add_brand_score_for_train_data(df)
    df = replace_item_condition_id_with_avg_price_for_train_dataset(df)
    df =  category_encoder_for_train_data_set(df)
    return df 

In [686]:
def feature_engineering_and_encoding_pipeline_for_test_data(df) : 
    df = sub_categories(df)
    df = add_name_score(df)
    df = add_brand_score_for_test_data(df,brand_scores_dict)
    df = replace_item_condition_id_with_avg_price_for_test_dataset(df,avg_prices_dict)
    df = category_encoder_for_test_data_set(df)
    return df 

In [687]:
df = feature_engineering_and_encoding_pipeline(df)
df2 = feature_engineering_and_encoding_pipeline_for_test_data(df2)

{1: 2.4499138988034623, 2: 3.4144498971521524, 3: 2.7745456348049453, 4: 0.0, 5: 10.0}


In [688]:
df.head()

,name,item_condition_id,brand_name,price,shipping,category1,category2,category3,name_score,brand_score,item_condition_id_score,category1_score,category2_score,category3_score
0,F/ship 4 Totoro Washi + 1 pen,1,other,12.0,1,Handmade,Paper Goods,Stationery,0.75,0.425335,2.449914,0.000000,0.737406,0.111839
1,UCLA Men's Bundle + Shorts,1,Adidas,76.0,1,Women,Other,Other,1.00,0.921929,2.449914,5.910399,2.326112,0.317467
2,Listing for lol,1,other,12.0,1,Beauty,Makeup,Lips,8.60,0.425335,2.449914,0.616439,1.624383,0.232257
3,25 pcs kawaii sticker flakes,1,other,3.0,1,Kids,Toys,Arts & Crafts,1.00,0.425335,2.449914,1.207762,1.914244,0.099438
4,Chanel Mini Lipgloss Set,2,Chanel,30.0,1,Beauty,Makeup,Lips,1.50,1.557531,3.414450,0.616439,1.624383,0.232257


In [689]:
df2.head()

,name,item_condition_id,brand_name,shipping,category1,category2,category3,name_score,brand_score,item_condition_id_score,category1_score,category2_score,category3_score
0,LuLaRoe TC Owls,1,LuLaRoe,1,Women,Jeans,Leggings,3.0,0.726222,2.449914,5.910399,2.450353,0.432127
1,Htc desire 626,2,HTC,0,Electronics,Cell Phones & Accessories,Cell Phones & Smartphones,3.0,0.940421,3.414450,10.000000,3.186006,1.802312
2,Vs Pink zip up jacket with hood,3,PINK,1,Women,Sweaters,Full Zip,-0.1,0.548231,2.774546,5.910399,2.530580,0.418824
3,Women's Adidas Cloudfoam Shoes,2,Adidas,0,Women,Shoes,Athletic,1.0,0.921929,3.414450,5.910399,4.341429,0.832277
4,Vs pink pants,2,PINK,0,Women,Athletic Apparel,Tracksuits & Sweats,2.3,0.548231,3.414450,5.910399,2.749697,0.520566


----------------------------------------------------------------
removing unneeded columns

In [690]:
df = df[['price','shipping', 'name_score', 'brand_score', 'item_condition_id_score', 'category1_score', 'category2_score','category3_score']]
df2 = df2[['shipping', 'name_score', 'brand_score', 'item_condition_id_score', 'category1_score', 'category2_score','category3_score']]

In [691]:
df.head()

,price,shipping,name_score,brand_score,item_condition_id_score,category1_score,category2_score,category3_score
0,12.0,1,0.75,0.425335,2.449914,0.000000,0.737406,0.111839
1,76.0,1,1.00,0.921929,2.449914,5.910399,2.326112,0.317467
2,12.0,1,8.60,0.425335,2.449914,0.616439,1.624383,0.232257
3,3.0,1,1.00,0.425335,2.449914,1.207762,1.914244,0.099438
4,30.0,1,1.50,1.557531,3.414450,0.616439,1.624383,0.232257


In [692]:
df2.head()

,shipping,name_score,brand_score,item_condition_id_score,category1_score,category2_score,category3_score
0,1,3.0,0.726222,2.449914,5.910399,2.450353,0.432127
1,0,3.0,0.940421,3.414450,10.000000,3.186006,1.802312
2,1,-0.1,0.548231,2.774546,5.910399,2.530580,0.418824
3,0,1.0,0.921929,3.414450,5.910399,4.341429,0.832277
4,0,2.3,0.548231,3.414450,5.910399,2.749697,0.520566


In [693]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.impute import SimpleImputer  # Added import for handling missing values

# Split the data into features (X) and the target variable (y)
X = df.drop('price', axis=1)
y = df['price']

# Split the data into training and validation sets (adjust the test_size as needed)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # You can choose a different strategy if needed

X_train = imputer.fit_transform(X_train)
X_val = imputer.transform(X_val)








In [694]:
# Linear Regression
linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, y_train)
linear_reg_predictions = linear_reg_model.predict(X_val)
linear_reg_mse = mean_squared_error(y_val, linear_reg_predictions)
linear_reg_rmse = np.sqrt(linear_reg_mse)
print("Linear Regression MSE:", linear_reg_mse)
print("Linear Regression RMSE:", linear_reg_rmse)

Linear Regression MSE: 880.9576991081893
Linear Regression RMSE: 29.680931574130035


In [695]:

# Random Forest Regression
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=0)  # You can adjust the parameters as needed
random_forest_model.fit(X_train, y_train)
random_forest_predictions = random_forest_model.predict(X_val)
random_forest_mse = mean_squared_error(y_val, random_forest_predictions)
random_forest_rmse = np.sqrt(random_forest_mse)
print("Random Forest Regression MSE:", random_forest_mse)
print("Random Forest Regression RMSE:", random_forest_rmse)

Random Forest Regression MSE: 1007.1168238373841
Random Forest Regression RMSE: 31.735103967647312


In [696]:
# SGD Regressor
sgd_regressor_model = SGDRegressor(max_iter=1000, tol=1e-3)  # You can adjust the parameters as needed
sgd_regressor_model.fit(X_train, y_train)
sgd_regressor_predictions = sgd_regressor_model.predict(X_val)
sgd_regressor_mse = mean_squared_error(y_val, sgd_regressor_predictions)
sgd_regressor_rmse = np.sqrt(sgd_regressor_mse)
print("SGD Regressor MSE:", sgd_regressor_mse)
print("SGD Regressor RMSE:", sgd_regressor_rmse)

SGD Regressor MSE: 889.1946040986146
SGD Regressor RMSE: 29.81936625917148
